In [ ]:
from __future__ import division
import os
import numpy as np
import time
import matplotlib.pyplot as plt
import scipy as scp
import pylab as pyl
import warnings
warnings.filterwarnings('ignore')
np.random.seed(1234)

%matplotlib inline 

%load_ext autoreload                                                                                                                                                                                            
%autoreload 

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
"""To compute distance matrix"""
def distmat( x, y ):
    return np.sum( x**2, 0 )[:,None] + np.sum( y**2, 0 )[None,:] - 2 * x.transpose().dot( y )

"""To Normalise a vector"""
normalize = lambda a: a/np.sum( a )

"""To Compute P"""
def GetP( u, K, v ):
    return u[:,None] * K * v[None,:]

def plotp( x, col, plt, scale = 200, edgecolors = "k" ):
  return plt.scatter( x[0,:], x[1,:], s = scale, edgecolors = edgecolors,  c = col, cmap = 'plasma', linewidths = 2 )

In [ ]:
def generate_data( N ):
    """
     N is a list of the size of the data on x and y
    """
    x = np.random.rand( 2, N[0] ) - 0.5
    theta = 2 * np.pi * np.random.rand( 1, N[1] )
    r = 0.8 + .2 * np.random.rand( 1, N[1] )
    y = np.vstack( ( r * np.cos( theta ), r * np.sin( theta ) ) )
    return x, y

In [ ]:
def spectral_decomposition( mat ):
    eig, v = np.linalg.eigh( mat )
    sorting_indices = np.argsort( eig )
    eig = eig[ sorting_indices ]
    v   = v[ : , sorting_indices ]
    
    print( "List of smallest eigenvalues: ", eig[:10] )
    print( "List of largest  eigenvalues: ", eig[-10:] )

    return eig, v



In [ ]:
import computational_OT

In [ ]:
N = [ 500, 600 ]
x, y = generate_data( N )

In [ ]:
a = normalize( np.ones( N[0] ) )
b = normalize( np.ones( N[1] ) )
# Log domain Sinkhorn
print("Log domain Sinkhorn.... ")
results_logSinkhorn = []
times_logSinkhorn   = []
logsinkhornP        = []
epsilons = [ 0.1 , 0.05 , 0.01, 0.005,  0.001]
#Cost matrix
C = distmat( x, y )
for eps in epsilons:

  print( "Sinkhorn for epsilon = "+str(eps)+":" )    
  print( "Doing for (",N[0], N[1],")." )
  print( " |- Iterating" )

  start = time.time()
  logsinkhorn = computational_OT.Log_domainSinkhorn( a, b, C, eps)
  output = logsinkhorn.update( niter = 500 )
  results_logSinkhorn.append( output )
  end = time.time()
  times_logSinkhorn.append( 1e3 * ( end - start ) )
  logsinkhornP.append( GetP( np.exp( output['potential_f']/eps ), np.exp( - C/eps ), np.exp(output['potential_g']/eps ) ) )

In [ ]:
plt.figure( figsize = ( 20, 7 ) )

plt.subplot( 2, 1, 1 ),
plt.title( "$||P1 -a||_1+||P1 -b||_1$" )
for i in range( len( results_logSinkhorn) ):
  error = np.asarray( results_logSinkhorn[i]['error'] )
  plt.plot( error, label = 'log-sinkhorn for $\epsilon = $'+ str(epsilons[i]) , linewidth = 2 )
plt.yscale( 'log' )
plt.legend()
plt.xlabel( "Iterations" )
plt.ylabel( "Error in log-scale" )
plt.show()


In [ ]:
!pip install perfplot

In [ ]:
import numpy as np
import perfplot
import numexpr as ne
epsilon = 0.001
n = 800

perfplot.show(
    setup = lambda n : ( np.random.rand( n, n ), np.random.rand( n, n ), np.random.rand( n, 1 ) ) ,
    kernels = [
        lambda A, B, x : np.dot( A, x ),
        lambda A, B, x : x * np.exp( - ( A )/epsilon ) ,
        lambda A, B, x : np.exp( - ( A )/epsilon ) ,
        lambda A, B, x : np.exp( - ( A )/epsilon, out = B ) ,
        lambda A, B, x : ne.evaluate( "exp(-(A/epsilon))" ),
        lambda A, B, x : ne.evaluate( "exp(-A)" ),
        ],
    labels = [ 
              "sinkhorn only the dot product",
              
              "log-sinkhorn: just vector matrix multiplication elementwise", 
              "log-sinkhorn: just the exponentiation",
              "log-sinkhorn: just the exponentiation with predefined space allocation",
              "using numexpr",
              "using numexpr but without epsilon",
              ],
    n_range = [ 10 ** k for k in range(4) ],
    xlabel = "len(a)",
    equality_check = False, 
)